# Notebook 03: Feature Engineering

**Objetivo**: Crear features temporales, de rezago y estadísticas para modelado predictivo de urgencias.

**⚠️ ANTI-LEAKAGE**: Todas las features usan SOLO información del pasado (shift aplicado donde necesario)

## Estrategia:
1. Cargar datos de urgencias redefinidas (Definición A: Percentil 75)
2. Filtrar productos con predictibilidad Moderada/Alta
3. Crear features **SIN DATA LEAKAGE**:
   - **Temporales**: semana, mes, trimestre, estacionalidad
   - **Lags**: rezagos de ventas (1, 2, 4, 8, 52 semanas)
   - **Rolling stats**: MA, std, min, max (con shift para evitar leakage)
   - **Urgency-specific**: días desde última urgencia, frecuencia
4. Preparar train/test split temporal
5. Guardar dataset listo para modelado

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Librerías cargadas")

## 1. Carga de Datos

In [ ]:
# Cargar urgencias redefinidas
df_urgencias = pd.read_csv('../data/simulated/urgencias_redefinidas.csv')
df_urgencias['week_start'] = pd.to_datetime(df_urgencias['week_start'])

# Cargar clasificación de productos
df_productos = pd.read_csv('../data/simulated/productos_predecibles.csv')

print(f"Datos cargados:")
print(f"  • Registros totales: {len(df_urgencias):,}")
print(f"  • Productos únicos: {df_urgencias['item_id'].nunique():,}")
print(f"  • Rango temporal: {df_urgencias['week_start'].min()} a {df_urgencias['week_start'].max()}")
print(f"\nDistribución de predictibilidad:")
print(df_productos['clasificacion'].value_counts())

In [ ]:
# Filtrar solo productos Predecibles y Moderados
productos_modelar = df_productos[
    df_productos['clasificacion'].isin(['Predecible', 'Moderado'])
]['item_id'].unique()

df = df_urgencias[df_urgencias['item_id'].isin(productos_modelar)].copy()
df = df.sort_values(['item_id', 'week_start']).reset_index(drop=True)

print(f"\n✓ Dataset filtrado:")
print(f"  • Productos a modelar: {len(productos_modelar):,}")
print(f"  • Registros: {len(df):,}")
print(f"  • Proporción urgencias: {df['is_urgent_a'].mean():.1%}")

## 2. Features Temporales

In [ ]:
# Extraer componentes temporales
df['year'] = df['week_start'].dt.year
df['month'] = df['week_start'].dt.month
df['quarter'] = df['week_start'].dt.quarter
df['week_of_year'] = df['week_start'].dt.isocalendar().week
df['day_of_year'] = df['week_start'].dt.dayofyear

# Features cíclicas (sin/cos para capturar periodicidad)
df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
df['week_sin'] = np.sin(2 * np.pi * df['week_of_year'] / 52)
df['week_cos'] = np.cos(2 * np.pi * df['week_of_year'] / 52)

# Indicadores especiales
df['is_q4'] = (df['quarter'] == 4).astype(int)
df['is_january'] = (df['month'] == 1).astype(int)

print("✓ Features temporales creadas")

## 3. Lag Features (Rezagos) - SIN LEAKAGE

In [ ]:
# Crear lags de ventas por producto
lag_periods = [1, 2, 4, 8, 52]  # 1w, 2w, 1m, 2m, 1y

for lag in lag_periods:
    df[f'sales_lag_{lag}'] = df.groupby('item_id')['total_sales'].shift(lag)
    print(f"  • Lag {lag} semanas creado")

# Lags de urgencias (binario) - YA usan shift, están bien
for lag in [1, 2, 4]:
    df[f'urgent_lag_{lag}'] = df.groupby('item_id')['is_urgent_a'].shift(lag)

print("\n✓ Lag features creadas (SIN data leakage)")

## 4. Rolling Statistics (Ventanas Móviles) - CRÍTICO: SIN LEAKAGE

In [ ]:
# CORRECCIÓN CRÍTICA: Rolling statistics calculadas sobre valores PASADOS
# Usar shift(1) primero para excluir la semana actual

windows = [4, 8, 12]

for window in windows:
    # Media móvil sobre ventas PASADAS
    df[f'roll_ma_{window}'] = df.groupby('item_id')['total_sales'].shift(1).transform(
        lambda x: x.rolling(window=window, min_periods=1).mean()
    )
    
    # Desviación estándar móvil sobre ventas PASADAS
    df[f'roll_std_{window}'] = df.groupby('item_id')['total_sales'].shift(1).transform(
        lambda x: x.rolling(window=window, min_periods=1).std()
    )
    
    # Mínimo móvil sobre ventas PASADAS
    df[f'roll_min_{window}'] = df.groupby('item_id')['total_sales'].shift(1).transform(
        lambda x: x.rolling(window=window, min_periods=1).min()
    )
    
    # Máximo móvil sobre ventas PASADAS
    df[f'roll_max_{window}'] = df.groupby('item_id')['total_sales'].shift(1).transform(
        lambda x: x.rolling(window=window, min_periods=1).max()
    )
    
    print(f"  • Rolling stats ventana {window} semanas (con shift)")

# Features derivadas - AHORA usan rolling sin leakage
df['sales_vs_roll_ma4'] = df['total_sales'] / (df['roll_ma_4'] + 1)
df['sales_vs_roll_ma12'] = df['total_sales'] / (df['roll_ma_12'] + 1)

print("\n✓ Rolling statistics creadas (SIN data leakage - shift aplicado)")

## 5. Features Específicas de Urgencias - SIN LEAKAGE

In [ ]:
# Días desde última urgencia - CORREGIDO
# Versión simple y clara que usa solo info pasada
df['days_since_urgent'] = 0

for producto in df['item_id'].unique():
    mask = df['item_id'] == producto
    df_prod = df[mask].copy()
    
    dias = []
    dias_acumulados = 0
    
    for idx in range(len(df_prod)):
        # Para la fila actual, ¿cuántos días desde la última urgencia?
        if idx == 0:
            dias.append(0)  # Primera semana, no hay historial
        else:
            # Mirar la semana ANTERIOR
            if df_prod.iloc[idx-1]['is_urgent_a'] == 1:
                # La semana pasada fue urgente, reiniciar contador
                dias_acumulados = 7  # Una semana = 7 días
            else:
                # La semana pasada no fue urgente, acumular
                dias_acumulados += 7
            dias.append(dias_acumulados)
    
    df.loc[mask, 'days_since_urgent'] = dias

# Frecuencia de urgencias en ventanas móviles - YA corregido antes
for window in [4, 8, 12]:
    df[f'urgent_freq_{window}w'] = df.groupby('item_id')['is_urgent_a'].shift(1).transform(
        lambda x: x.rolling(window=window, min_periods=1).sum()
    )
    df[f'urgent_rate_{window}w'] = df[f'urgent_freq_{window}w'] / window

print("✓ Features de urgencias creadas (SIN data leakage)")
print(f"  • days_since_urgent: promedio {df['days_since_urgent'].mean():.1f} días")
print(f"  • urgent_rate_4w: promedio {df['urgent_rate_4w'].mean():.2%}")
print(f"\n⚠️  IMPORTANTE: TODAS las features usan solo información PASADA")

## 6. Features de Tendencia

In [ ]:
# Diferencias (cambio respecto a semana anterior)
df['sales_diff_1'] = df.groupby('item_id')['total_sales'].diff(1)
df['sales_diff_4'] = df.groupby('item_id')['total_sales'].diff(4)

# Tasa de crecimiento
df['sales_pct_change_1'] = df.groupby('item_id')['total_sales'].pct_change(1)
df['sales_pct_change_4'] = df.groupby('item_id')['total_sales'].pct_change(4)

# Momentum (diferencia entre MA corta y larga)
df['momentum'] = df['roll_ma_4'] - df['roll_ma_12']

print("✓ Features de tendencia creadas")

## 7. Limpieza y Preparación Final

In [ ]:
# Revisar valores faltantes
print("Missing values por columna:")
missing = df.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)
print(missing.head(10))

# Rellenar missing values
lag_cols = [col for col in df.columns if 'lag_' in col or 'roll_' in col or 'urgent_freq' in col or 'urgent_rate' in col]
df[lag_cols] = df.groupby('item_id')[lag_cols].ffill()

# Para diferencias y pct_change: rellenar con 0
diff_cols = [col for col in df.columns if 'diff' in col or 'pct_change' in col or 'momentum' in col]
df[diff_cols] = df[diff_cols].fillna(0)

# Reemplazar infinitos por NaN y luego rellenar
df = df.replace([np.inf, -np.inf], np.nan)
df = df.fillna(0)

print(f"\n✓ Missing values manejados")
print(f"  • Nulls restantes: {df.isnull().sum().sum()}")

## 8. Análisis de Correlaciones

In [ ]:
# Seleccionar features numéricas relevantes
feature_cols = [col for col in df.columns if col not in [
    'item_id', 'week_start', 'week_id', 'week_num', 'total_sales', 
    'is_urgent_a', 'is_urgent_b', 'is_urgent_c',
    'threshold_a', 'threshold_b', 'threshold_c', 'mean_prod', 'std_prod',
    'category', 'dept', 'ma_4', 'mes'  # Excluir ma_4 del notebook 02
]]

# Calcular correlación con variable objetivo
correlations = df[feature_cols + ['is_urgent_a']].corr()['is_urgent_a'].drop('is_urgent_a')
correlations = correlations.sort_values(ascending=False)

print("\n📊 TOP 15 FEATURES MÁS CORRELACIONADAS CON URGENCIA:")
print(correlations.head(15))

print("\n📊 BOTTOM 10 FEATURES (menor correlación):")
print(correlations.tail(10))

In [ ]:
# Visualizar top correlaciones
fig, ax = plt.subplots(figsize=(10, 8))
top_corr = correlations.head(20)
top_corr.plot(kind='barh', ax=ax, color='steelblue')
ax.set_xlabel('Correlación con is_urgent_a')
ax.set_title('Top 20 Features por Correlación con Urgencias')
ax.axvline(x=0, color='black', linestyle='--', linewidth=0.8)
plt.tight_layout()
plt.savefig('../results/feature_correlations.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Gráfico guardado: results/feature_correlations.png")

## 9. Train/Test Split Temporal

In [ ]:
# Split temporal: últimas 26 semanas (6 meses) para test
fechas_unicas = sorted(df['week_start'].unique())
fecha_split = fechas_unicas[-26]

df_train = df[df['week_start'] < fecha_split].copy()
df_test = df[df['week_start'] >= fecha_split].copy()

print(f"\n📅 SPLIT TEMPORAL:")
print(f"\nTRAIN:")
print(f"  • Fechas: {df_train['week_start'].min()} a {df_train['week_start'].max()}")
print(f"  • Semanas: {df_train['week_start'].nunique()}")
print(f"  • Registros: {len(df_train):,}")
print(f"  • Urgencias: {df_train['is_urgent_a'].sum():,} ({df_train['is_urgent_a'].mean():.1%})")

print(f"\nTEST:")
print(f"  • Fechas: {df_test['week_start'].min()} a {df_test['week_start'].max()}")
print(f"  • Semanas: {df_test['week_start'].nunique()}")
print(f"  • Registros: {len(df_test):,}")
print(f"  • Urgencias: {df_test['is_urgent_a'].sum():,} ({df_test['is_urgent_a'].mean():.1%})")

## 10. Guardar Datasets Preparados

In [ ]:
# Guardar dataset completo con features (SOBRESCRIBE)
df.to_csv('../data/simulated/dataset_features.csv', index=False)
print(f"✓ Dataset completo guardado: data/simulated/dataset_features.csv")
print(f"  • Shape: {df.shape}")

# Guardar train/test (SOBRESCRIBE)
df_train.to_csv('../data/simulated/train_features.csv', index=False)
df_test.to_csv('../data/simulated/test_features.csv', index=False)
print(f"\n✓ Train/Test guardados:")
print(f"  • train_features.csv: {df_train.shape}")
print(f"  • test_features.csv: {df_test.shape}")

# Guardar lista de features para modelado (SOBRESCRIBE)
features_modeling = [col for col in feature_cols if col in df.columns]
pd.DataFrame({'feature': features_modeling}).to_csv('../data/simulated/feature_list.csv', index=False)
print(f"\n✓ Lista de features guardada: {len(features_modeling)} features")

print(f"\n⚠️  ARCHIVOS ANTERIORES SOBRESCRITOS (sin data leakage)")

## 11. Resumen Final

In [ ]:
print("\n" + "="*70)
print("📊 RESUMEN FEATURE ENGINEERING")
print("="*70)

print(f"\n✓ Productos procesados: {df['item_id'].nunique():,}")
print(f"✓ Registros totales: {len(df):,}")
print(f"✓ Features creadas: {len(feature_cols)}")

print(f"\n📂 ARCHIVOS GENERADOS:")
print(f"  • dataset_features.csv - Dataset completo con todas las features")
print(f"  • train_features.csv - Training set ({len(df_train):,} registros)")
print(f"  • test_features.csv - Test set ({len(df_test):,} registros)")
print(f"  • feature_list.csv - Lista de features para modelado")
print(f"  • feature_correlations.png - Visualización de correlaciones")

print(f"\n🎯 CARACTERÍSTICAS DEL DATASET:")
print(f"  • Target: is_urgent_a (Definición A: Percentil 75)")
print(f"  • Balance: {df['is_urgent_a'].mean():.1%} urgencias")
print(f"  • Horizonte temporal: {df['week_start'].nunique()} semanas")
print(f"  • Split: {len(df_train)} train / {len(df_test)} test")

print(f"\n✅ DATASET LISTO PARA MODELADO (SIN DATA LEAKAGE)")
print(f"\n🔒 GARANTÍA ANTI-LEAKAGE:")
print(f"  ✓ Lags de ventas: shift() aplicado")
print(f"  ✓ Rolling stats: shift() aplicado")
print(f"  ✓ Urgency features: shift() aplicado")
print(f"  ✓ days_since_urgent: solo usa info pasada")
print(f"\nMétricas esperadas en XGBoost: 70-85% accuracy (NO 99%)")
print("="*70)